# Import

In [1]:
import json
import pandas as pd
import re
import time
from functools import partial
from module import unicode
from symspellpy import SymSpell, Verbosity
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rhinoWrapper import Rhino
rn = Rhino()

filepath:  /home/faraway/.local/lib/python3.7/site-packages
classpath:  /home/faraway/.local/lib/python3.7/site-packages/rhinoMorph/lib/rhino.jar
Constructing Dictionaries...
Current mode: python	Current path: /home/faraway/.local/lib/python3.7/site-packages/rhinoMorph/resource/
RHINO started!
Constructing Dictionaries Completed.



# load

## encoding

In [2]:
# Encoding
encoding= "utf-8-sig"

## Correct

In [3]:
corr_path = "dictionary/symspell_2.txt"

## Grammar, Vocabulary

In [53]:
with open("dictionary/gram_vocab_dict_2.json", 'r', encoding= "utf-8-sig") as outfile:
    gram_vocab_dict = json.load(outfile)

gram_list = gram_vocab_dict["문법"]
vocab_list = gram_vocab_dict["어휘"]
grammar_df = pd.read_csv("dictionary/문법.csv", index_col= "전체 번호", encoding= encoding)
vocabulary_df = pd.read_csv("dictionary/어휘_4_foreign_vocab_with_clear_means.csv", index_col= "전체 번호", encoding= encoding)
gram_tokenizer = unicode.split_syllables
vocab_tokenizer = partial(rn.wholeResult_list,
                          pos= ["NNG", "NNP", "NNB", "NP", "NR", "VV", "VA", "MAG", "MM"])

/home/faraway/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (12,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Similarity

In [5]:
# similar_tokenizer = partial(rn.onlyMorph_list, pos= ["NNG", "NNP", "VV", "VA", "XR"])
similar_tokenizer = rn.onlyMorph_list

w2v_path = "recommend/word2vec/recommend_word2vector.model"
ft_path = 'recommend/fasttext/recommend_fasttext.model'
qna_level_path = 'recommend/aihub_qna_level_utf8sig.csv'
question_list_path = 'recommend/aihub_question_utf8sig.txt'

# Sub Classes

## 1-1. Correct

In [6]:
class Correct :
    def __init__(self, dict_path) -> None:
        self.sym = SymSpell()
        print("Loading Symspell dictionary ...")
        self.sym.load_dictionary(dict_path, 
                                term_index= 0, count_index= 1, 
                                separator= "\t",
                                encoding= encoding)
        print("End")
        

        
    # 파이프라인용
    def correction(self, sentence):
        segment = sentence.split(' ')

        corrected = []
        for word in segment:
            term = unicode.split_syllables(word)
            suggetion = self.sym.lookup(term, Verbosity.ALL, max_edit_distance=2)

            if len(suggetion) == 0:
                corrected.append(word)
            else:
                word = unicode.join_jamos(suggetion[0].term)
                corrected.append(word)
        
        return ' '.join(corrected)

### instance

In [7]:
correct = Correct(dict_path = corr_path)

Loading Symspell dictionary ...
End


In [8]:
# 오타 있는 문장
paragraph = '저는 종국에서 왔습니다 한국 온 이유는 저는 개인적으로 한긱 음식이나 그리고 한긱 영화 한긱 두라마 좋와해서 한국에 왔습니다.'
correct.correction(paragraph)

'저는 중국에서 왔습니다 한국 온 이유는 저는 개인적으로 한국 음식이나 그리고 한국 영화 한국 드라마 좋아해서 한국에 왔습니다.'

In [9]:
# 오타 없는 문장
# 그대로 나와야 함
paragraph = '제가 필라테스라는 취미를 가지게 된 이유는 필라테스를 하면 우선 다이어트 되고 또한 자세를 교정할 수 있습니다 또한 필라테스를 하면 매우 개운해져서 즐겁습니다'
correct.correction(paragraph)

'제가 필라테스라는 취미를 가지게 된 이유는 필라테스를 하면 우선 다이어트 되고 또한 자세를 교정할 수 있습니다 또한 필라테스를 하면 매우 개운해져서 즐겁습니다'

## 1-2. Correct_2

In [10]:
class Correct_2 :       # [NUM] 토큰으로 마스킹하여 [NUM] 학년
    def __init__(self, dict_path) -> None:
        self.sym = SymSpell()
        print("Loading Symspell dictionary ...")
        self.sym.load_dictionary(dict_path, 
                                term_index= 0, count_index= 1, 
                                separator= "\t",
                                encoding= encoding)
        print("End")
        self.correct_method = Correct_Method()
        

        
    # 파이프라인용
    def correction(self, sentence):
        
        sentence, nums = self.correct_method.preprocess(sentence)
        segment = sentence.split(' ')

        corrected = []
        for word in segment:
            term = unicode.split_syllables(word)
            suggetion = self.sym.lookup(term, Verbosity.ALL, max_edit_distance=2)

            if len(suggetion) == 0:
                corrected.append(term)
            else:
                word = suggetion[0].term
                corrected.append(word)
        result = ' '.join(corrected)
        
        result = self.correct_method.postprocess([result, nums])
        return result
    

class Correct_Method :
    def __init__(self) :
        self.NUM_TOKEN = "[NUM]"
    
    def preprocess(self, text) :
        filtered_syllables = re.sub(r'[^가-힣0-9ㄱ-ㅎㅏ-ㅣ\?\!\. ]', ' ', text)
        syllables = unicode.split_syllables(filtered_syllables) 
        
        masked_num = re.findall(r'\d+', syllables)      # 숫자만 찾는다
        
        for num in masked_num :
            syllables = syllables.replace(num, self.NUM_TOKEN, 1)      # 숫자 마스킹 # [NUM]학년 [NUM]반
            
        syllables = ' '.join(syllables.split())
        inform = (syllables, masked_num)        # ([[NUM]ㅎㅏㄱㄴㅕㄴ [NUM]ㅂㅏㄴ], [1, 2])
        return inform    # 초중성 나눠진 어절단위 문장, 숨겨진 숫자
    
    def postprocess(self, inform) :
        assert len(inform) == 2
        syllables = inform[0]
        masked_num = inform[1]
        
        for num in masked_num :         # ([[NUM]ㅎㅏㄱㄴㅕㄴ [NUM]ㅂㅏㄴ], [1, 2]) -> 1학년 2반
            syllables = syllables.replace(self.NUM_TOKEN, num, 1)
        
        text = unicode.join_jamos(' '.join(syllables.split())).strip()
        
        return text

### instance

In [11]:
correct_2 = Correct_2(dict_path = corr_path)

Loading Symspell dictionary ...
End


In [12]:
# 오타 있는 문장
paragraph = '저는 종국에서 왔습니다 한국 온 이유는 저는 개인적으로 한긱 음식이나 그리고 한긱 영화 한긱 두라마 좋와해서 한국에 왔습니다.'
correct_2.correction(paragraph)

'저는 중국에서 왔습니다 한국 온 이유는 저는 개인적으로 한국 음식이나 그리고 한국 영화 한국 드라마 좋아해서 한국에 왔습니다.'

In [13]:
# 오타 없는 문장
# 그대로 나와야 함
paragraph = '제가 필라테스라는 취미를 가지게 된 이유는 필라테스를 하면 우선 다이어트 되고 또한 자세를 교정할 수 있습니다 또한 필라테스를 하면 매우 개운해져서 즐겁습니다'
correct_2.correction(paragraph)

'제가 필라테스라는 취미를 가지게 된 이유는 필라테스를 하면 우선 다이어트 되고 또한 자세를 교정할 수 있습니다 또한 필라테스를 하면 매우 개운해져서 즐겁습니다'

In [14]:
correct.correction("1하년 2바")

'하는 바'

In [15]:
correct_2.correction("1하년 2바")

'1학년 2차'

## 2. Grammar

In [16]:
class Grammar :
    def __init__(self, tokenizer, gram_list, gram_df) -> None:
        self.tokenizer = tokenizer
        # 초중종성 나누기
        self.gram_list = [[x[0], self.tokenizer(x[1])] for x in gram_list]
        
        self.gram_df = gram_df
        
    def mark_grammar(self, text, return_columns = False) :
        """
        1. 문장에서 문법에 해당하는 추려낸다
        2. 추려낸 문법의 시작과 끝 인덱스를 구한다
        3. "-" 기호가 앞이나 뒤에 붙은 경우 이어지는 문자가 있어야 한다 (빈 문자이면 안된다) 없으면 제외
        """
        def check_pos(text_base, text_check) :
            """
            특정 문장 내에 위치한 단어의 시작과 끝 인덱스 리스트 반환
            text_base : 특정 문장
            text_check : 문장에서 찾아낼 단어
            
            return (시작_인덱스, 끝_인덱스)
            """
            copy_text_check = re.sub(r"[-*]", "", text_check)
            
            text_len = len(copy_text_check)
            text_pos = []
            index = -1
            while True:
                index = text_base.find(copy_text_check, index + 1)
                if index >= 0 :
                    text_pos.append((index, index + text_len))
                elif index == -1:
                    break
                else :
                    assert True

            return text_pos
        
        text_re = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣ ]", " ", text)     # 한글이 아니면 전부 빈칸 처리
        text_syllables = self.tokenizer(text_re)
        
        word_columns = []
        for key, word in self.gram_list :         # 문법 리스트 요소 하나하나 체크
            key_columns = key - 1
            pos_index_list = check_pos(text_syllables, word)
            if len(pos_index_list) > 0 :        # 문장내에서 찾지 못한 경우 넘어감
                # 문장의 앞 뒤를 체크할지 여부
                is_check_front = True if word[0] == "-" else False
                is_check_back = True if word[-1] == "-" else False
                
                checked_pos_index = []
                for pos_index in pos_index_list :
                    #(시작_인덱스, 끝_인덱스)
                    is_correct = True
                    if is_check_front :
                        if pos_index[0] == 0 or text_syllables[pos_index[0] - 1] == " " :       # 문장의 앞이 빈칸이면 False
                            is_correct = False
                    if is_check_back and is_correct :       # is_check_front에서 is_correct = False가 되면 할 필요없음
                        if pos_index[1] == len(text_syllables) or text_syllables[pos_index[1]] == " " :         # 문장의 뒤가 빈칸이면 False
                            is_correct = False
                    
                    if is_correct :
                        checked_pos_index.append(pos_index)
                        
                if checked_pos_index != []:
                    if len(word_columns) <= 0 :        # word_columns 에 없는 경우
                        word_columns.append((word, key_columns))
                    else :
                        is_add = True
                        for word_in_words, key_in_words in word_columns :
                            if word == word_in_words :      # 동음이의어 -> 급수 비교
                                if int(self.gram_df.iloc[key_in_words, :]["등급"][0]) <= int(self.gram_df.iloc[key_columns, :]["등급"][0]) :
                                    # 저장된 word_columns 내부의 "등급"의 수가 더 작거나 같은 경우 그대로
                                    is_add = False
                                    continue
                            elif key_columns == key_in_words :
                                is_add = False
                                continue
                                
                            is_word_in_wiw = True if word_in_words.find(word) != -1 else False       # word가 word_columns 내부의 단어 내의 단어 인 경우
                            is_wiw_in_word = True if word.find(word_in_words) != -1 else False       # word 안에 word_columns 내부의 단어가 들어있는 경우
                            # print(word, word_in_words, is_word_in_wiw, is_wiw_in_word)
                            
                            if not is_word_in_wiw and not is_wiw_in_word :      # 아예 다른 단어
                                continue
                            if is_word_in_wiw :
                                is_add = False
                                break
                            elif is_wiw_in_word :
                                word_in_words = word
                                key_in_words = key_columns
                                is_add = False
                                break
                            else :
                                assert False
                                
                        if is_add :
                            word_columns.append((word, key_columns))
        
        if return_columns :
            return word_columns
        return word_columns
    
    def rate_grammar(self, text) :
        text_split = text.split(' ')
        text_filter = list(filter(None, text_split))
        
        mark_grammar_partial = partial(self.mark_grammar,
                                    return_columns = True)
        marking = list(map(mark_grammar_partial, text_filter))
        
        checker = []
        for mark in marking :
            for part in mark :
                checker.append(part[1])
        
        # (겹치는 문법을 제외한 문장의 등급의 합) / (전체 문법 수) 
        all_num = len(checker)
        if all_num <= 0 :
            return 0
        
        only_checker = list(set(checker))
        grade = self.gram_df.iloc[only_checker, :]["등급"].replace(r"[급]", "", regex= True).astype(int).sum() / all_num
        
        return grade

### instance

In [17]:
grammar = Grammar(tokenizer= gram_tokenizer, 
                  gram_list= gram_list,
                  gram_df= grammar_df)

In [18]:
grammar.mark_grammar("개미는 뚠뚠 오늘도 뚠뚠 열심히 일을 하네 뚠뚠")

[]

In [19]:
grammar.rate_grammar("개미는 뚠뚠 오늘도 뚠뚠 열심히 일을 하네 뚠뚠")

0

## 3. Vocabulary

In [20]:
class Vocabulary :
    def __init__(self, tokenizer, vocab_list, vocab_df) :
        self.tokenizer = tokenizer
        self.vocab_dict = {x[1] : x[0] for x in vocab_list}
        self.vocab_df = vocab_df
        
    def mark_vocabulary(self, text) :
        text_re = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣ ]", "", text)     # 한글이 아니면 전부 빈칸 처리
        text_tokens = self.tokenizer(text_re)
        
        token_keys = list(set(map(lambda x : x - 1, filter(None, [self.vocab_dict.get(x[0]) for x in text_tokens]))))
        
        return token_keys
    
    # 등급별 어휘 사전 참고
    # 등급 책정 함수

    def rate_vocabulary(self, text) :
        mark = self.mark_vocabulary(text)
        temp_text = self.vocab_df.iloc[mark].drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
        temp_grade = temp_text['등급'].str.replace('급', '').to_frame().apply(pd.to_numeric)
        
        if len(temp_grade) == 0 :
            return 0
        grade = temp_grade.sum()/len(temp_grade)
        return grade['등급']

### instance

In [21]:
vocabulary = Vocabulary(tokenizer= vocab_tokenizer,
                        vocab_list= vocab_list,
                        vocab_df= vocabulary_df)

In [22]:
vocabulary.mark_vocabulary('저는 미국에서 왔습니다 저는 한국에서 있+ 오는 이유는 한국어를 제대로 공부하고 싶어서 왔습니다')

[554, 717, 533, 3415, 249, 3167, 478, 575]

In [23]:
vocabulary.rate_vocabulary('저는 미국에서 왔습니다 저는 한국에서 있+ 오는 이유는 한국어를 제대로 공부하고 싶어서 왔습니다')

1.5

## 4-1. Similarity

In [24]:
class Similarity :
    def __init__(self, tokenizer,   # 토크나이저
                 w2v_path, ft_path,     # 임베딩 모델
                 qna_level_path, question_list_path     # 문제, 질문 리스트
                 ) :
        self.similarity_method = Similarity_Method(tokenizer= tokenizer,
                                                   w2v_path= w2v_path, ft_path= ft_path)
        
        # 'aihub_qna_level.csv'에는 질문(Question) 답변(AnswerLabelText), 등급(SentenceSpeechLV), 질문라벨코드(question_label), 총 4가지 열
        # 모두 '상'등급 문장
        self.qna_data = pd.read_csv(qna_level_path, encoding=encoding)
        # 라벨 순서대로 나열된 질문 리스트
        with open(question_list_path, 'r', encoding=encoding) as file:
            self.question_list = [line for line in file.read().splitlines()]
        
    def recommend(self, ques, student_answer):

        # print('질문 : ', ques)
        # print('학습자 답변 : ', student_answer, '\n')

        # database : 같은 질문의 '상' 등급 답변 모음
        question_idx = self.question_list.index(ques)
        database = self.qna_data[self.qna_data['question_label']==question_idx]
        database = database['AnswerLabelText'].reset_index(drop=True)
        database = database.tolist()

        # 3가지 유사도 기법에 넣어 가장 유사도가 높은 문장(*_ans) 찾기 (*_sim은 유사도)
        _, w2v_ans = self.similarity_method.W2Vresult(student_answer, database)
        _, ft_ans = self.similarity_method.FTresult(student_answer, database)
        _, cs_ans = self.similarity_method.CSresult(student_answer, database)

        # 중복 답변 제거
        answer = set([w2v_ans, ft_ans, cs_ans])

        return answer
    
class Similarity_Method :
    def __init__(self, tokenizer, w2v_path, ft_path) :
        self.tokenizer = tokenizer
        self.load_w2v = Word2Vec.load(w2v_path)
        self.load_ft = Word2Vec.load(ft_path)

    def W2Vresult(self, sen, senDB):
        sen_tokens = set(self.tokenizer(sen))    # <학습자 문장> 토큰화
        if len(sen_tokens) == 0:
            sen_tokens = sen
        
        sen_list = []       # '상'등급 문장들 토큰화해서 리스트에 담기
        for sentence in senDB:
            sen_list.append(set(self.tokenizer(sentence)))

        sim_list = []       # <학습자 문장>과 '상'등급 문장들 간의 유사도 구해서 담기
        for sen_token in sen_list:
            similarity = self.load_w2v.wv.n_similarity(sen_tokens, sen_token)
            sim_list.append(similarity)

        result = max(sim_list)
        result_sentence = senDB[sim_list.index(result)]      # 최대 유사도 값을 가진 문장 구하기

        return result, result_sentence
    
    # word2vec과 방식은 같음
    def FTresult(self, sen, senDB):
        sen_tokens = set(self.tokenizer(sen))
        if len(sen_tokens) == 0:
            sen_tokens = sen
        
        sen_list = []
        for sentence in senDB:
            sen_list.append(set(self.tokenizer(sentence)))

        sim_list = []
        for sen_token in sen_list:
            similarity = self.load_ft.wv.n_similarity(sen_tokens, sen_token)
            sim_list.append(similarity)
        
        result = max(sim_list)
        result_sentence = senDB[sim_list.index(result)]

        return result, result_sentence
    
    def CSresult(self, sen, senDB):
        sen_tokens = set(self.tokenizer(sen))
        if len(sen_tokens) == 0:
            sen_tokens = sen
        sen_tf = ' '.join(sen_tokens)     # <학습자 문장 sen> rhino 형태소 분석기로 토큰화된 것을 ' '안에 다 넣어줘야 함 (개수 1개로 줄여야함)

        tokens_list = [sen_tf]       # 리스트 1개로 '상'등급 문장과 묶어줌 (tokens_list[0]이 <학습자 문장>)
        for sen in senDB:
            combine_token = ' '.join(set(self.tokenizer(sen)))
            tokens_list.append(combine_token)

        tfidfv = TfidfVectorizer().fit(tokens_list)     # TF-IDF matrix 생성(행 : 문장들, 열 : 토큰화된 단어들의 tf-idf 값)
        tfidf_matrix = tfidfv.transform(tokens_list).toarray()

        cos = cosine_similarity(tfidf_matrix, tfidf_matrix)     # 코사인 유사도 matrix 생성 (왼쪽대각선 기준 대칭인 행렬)

        cos = cos[0].tolist()       # 첫 번째 문장이 <학습자 문장>, 최대값 찾게 리스트로 변환
        result = max(cos[1:])   # cos[0]은 자기 자신에 대한 유사도 값이므로 항상 1이니, 그 이후 값에서 최대값을 찾기
        result_sentence = senDB[cos.index(result)-1]    # <학습자 문장> 포함 안 된(-1) senDB에서 인덱스 이용해 문장 찾기
        
        return result, result_sentence

### instance

In [25]:
similarity = Similarity(tokenizer= similar_tokenizer, w2v_path= w2v_path, ft_path= ft_path,
                        qna_level_path= qna_level_path, question_list_path= question_list_path)

In [26]:
similarity.recommend("여러분 나라의 학교와 한국의 학교는 무엇이 가장 다릅니까?", "오늘")

{'아마 공부하는 시간이 제일 다른 거 같습니다 러시아에서 어/ 아침부터 뭐/ 일곱 시 뭐/ 여덟시부터 어/ 오후 한 시까지 시간 공부하는 시간이면 한국에서는 거의 하루 종일 공부하는 거 같습니다',
 '여러분 나라의 학교와 한국의 학교는 무엇이 가장 다릅니까 우리 나라의 학교와 한국의 학교의 다른 점은 수업 지난 시간이 다르는 것입니다 우리 나라 학교는 하루 종일 다녀야 되고 한국의 학교는 오전 반과 오후 반이 따로 있다고 들었습니다',
 '한국의 학교는 저희 나라 학+ 학교보다는 공부를 더 많이 시키고 교과목 이외에 어/ 다른 활동 시간이 적은 거 같습니다'}

In [27]:
q = '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요? 왜 그 방법이 가장 도움이 되었나요?'
a = '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요 드라마 보고 대화 듣기 이 왜 그 방법이 가장 도움 되었어요 듣기 연습하고 새로운 어휘 나왔어요'

In [28]:
similarity.recommend(q, a)

{'저는 한국어를 배우면서 가장 도움이 된 방법은 드라마나 영화를 보는 거라고 생각해요 따라서 말하기도 연습할 수 있고 새로운 표현과 어휘도 익힐 수 있고 또 듣기도 연습해서 이 방법은 가장 좋은 방법이에요',
 '한국어를 배우면서 가장 도움이 되었던 방법은 드라마를 많이 배우고 드라마에서 나오는 대화를 어/ 직접 하면서 말하기나 듣기를 연습하는 것입니다 그 이유는 어/ 재밌는 방법으로 연습할 수 있기 때문입니다'}

## 4-2. Similarity_Only

In [29]:
class Similarity_Only :
    def __init__(self, tokenizer,   # 토크나이저
                 w2v_path, ft_path,     # 임베딩 모델
                 qna_level_path, question_list_path     # 문제, 질문 리스트
                 ) :
        self.similarity_method = Similarity_Method_Only(tokenizer= tokenizer,
                                                        w2v_path= w2v_path, ft_path= ft_path)
        
        # 'aihub_qna_level.csv'에는 질문(Question) 답변(AnswerLabelText), 등급(SentenceSpeechLV), 질문라벨코드(question_label), 총 4가지 열
        # 모두 '상'등급 문장
        self.qna_data = pd.read_csv(qna_level_path, encoding=encoding)
        # 라벨 순서대로 나열된 질문 리스트
        with open(question_list_path, 'r', encoding=encoding) as file:
            self.question_list = [line for line in file.read().splitlines()]
        
    def recommend(self, ques, student_answer, similar_up = 0.85):

        print('질문 : ', ques)
        print('학습자 답변 : ', student_answer, '\n')

        # database : 같은 질문의 '상' 등급 답변 모음
        question_idx = self.question_list.index(ques)
        database = self.qna_data[self.qna_data['question_label']==question_idx]
        database = database['AnswerLabelText'].reset_index(drop=True)
        database = database.tolist()

        # 3가지 유사도 기법에 넣어 가장 유사도가 높은 문장(*_ans) 찾기 (*_sim은 유사도)
        _, w2v_ans = self.similarity_method.W2Vresult(student_answer, database, similar_up= similar_up)
        
        result = list(set(w2v_ans))
        return result
    
class Similarity_Method_Only :
    def __init__(self, tokenizer, w2v_path, ft_path) :
        self.tokenizer = tokenizer
        self.load_w2v = Word2Vec.load(w2v_path)
        self.load_ft = Word2Vec.load(ft_path)

    # 파이프라인용
    def W2Vresult(self, sen, senDB, similar_up = 0.85):
        sen_tokens = set(self.tokenizer(sen))
        if len(sen_tokens) == 0:
            return [], []
        
        sim_list = []
        for sentence in senDB:
            sen_token = set(self.tokenizer(sentence))
            similarity = self.load_w2v.wv.n_similarity(sen_tokens, sen_token)
            sim_list.append(similarity)

        sentence_similarities = sorted(sim_list)

        recommend_similarities = []
        recommend_sentences = []
        for i in range(3) :
            similarity = sentence_similarities[-(i + 1)]
            print(similarity)       # 추천 문장과의 유사도
            if similarity > similar_up :      # 유사도가 이 값 보다 크다면..
                recommend_similarities.append(similarity)
                recommend_sentence = senDB[sim_list.index(similarity)]
                recommend_sentences.append(recommend_sentence)
            else :
                break

        return recommend_similarities, recommend_sentences
    
    # word2vec과 방식은 같음
    def FTresult(self, sen, senDB, similar_up):
        sen_tokens = set(self.tokenizer(sen))
        if len(sen_tokens) == 0:
            return [], []
        
        sim_list = []
        for sentence in senDB:
            sen_token = set(self.tokenizer(sentence))
            similarity = self.load_ft.wv.n_similarity(sen_tokens, sen_token)
            sim_list.append(similarity)

        sentence_similarities = sorted(sim_list)

        recommend_similarities = []
        recommend_sentences = []
        for i in range(3) :
            similarity = sentence_similarities[-(i + 1)]
            if similarity > similar_up :      # 유사도가 이 값 보다 크다면..
                recommend_similarities.append(similarity)
                recommend_sentence = senDB[sim_list.index(similarity)]
                recommend_sentences.append(recommend_sentence)
            else :
                break

        return recommend_similarities, recommend_sentences

### instance

In [30]:
similarity_only = Similarity_Only(tokenizer= similar_tokenizer, w2v_path= w2v_path, ft_path= ft_path,
                        qna_level_path= qna_level_path, question_list_path= question_list_path)

In [31]:
similarity_only.recommend("여러분 나라의 학교와 한국의 학교는 무엇이 가장 다릅니까?", "오늘")

질문 :  여러분 나라의 학교와 한국의 학교는 무엇이 가장 다릅니까?
학습자 답변 :  오늘 

0.55410445


[]

In [32]:
q = '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요? 왜 그 방법이 가장 도움이 되었나요?'
a = '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요 드라마 보고 대화 듣기 이 왜 그 방법이 가장 도움 되었어요 듣기 연습하고 새로운 어휘 나왔어요'

similarity_only.recommend(q, a, similar_up= 0.85)

질문 :  한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요? 왜 그 방법이 가장 도움이 되었나요?
학습자 답변 :  한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요 드라마 보고 대화 듣기 이 왜 그 방법이 가장 도움 되었어요 듣기 연습하고 새로운 어휘 나왔어요 

0.97970545
0.97911143
0.9684288


['한국어를 배우면서 가장 도움이 되었던 방법은 드라마에서 나온 그/ 자막을 보면서 그대로 따라 어/ 읽 예 연습하는 것입니다 그것은 실제로 뭐/ 말하는 것을 대화하는 것을 연습할 수 있기 때문입니다',
 '한국어를 배우면서 가장 도움이 되었던 방법은 드라마를 많이 배우고 드라마에서 나오는 대화를 어/ 직접 하면서 말하기나 듣기를 연습하는 것입니다 그 이유는 어/ 재밌는 방법으로 연습할 수 있기 때문입니다',
 '저는 한국어를 배우면서 가장 도움이 된 방법은 드라마나 영화를 보는 거라고 생각해요 따라서 말하기도 연습할 수 있고 새로운 표현과 어휘도 익힐 수 있고 또 듣기도 연습해서 이 방법은 가장 좋은 방법이에요']

# Integrate Classes

## 1. Gram_Vocab_Rater

In [33]:
class Gram_Vocab_Rater :
    def __init__(self, correct, grammar, vocabulary) :
        self.correct = correct
        self.grammar = grammar
        self.vocabulary = vocabulary
         
    def rate(self, text) :
        text_correct = correct.correction(text)
        text_split = text_correct.strip().split(' ')
        text_filter = list(filter(None, text_split))
        text_join = ' '.join(text_filter)
        
        result = {
            "text" : text_join,
            "grammar" : grammar.rate_grammar(text),
            "vocabulary" : vocabulary.rate_vocabulary(text)
        }
        
        return result

### instance

In [34]:
rater = Gram_Vocab_Rater(correct= correct, 
                         grammar= grammar, 
                         vocabulary= vocabulary)

s = '저는 종국에서 왔습니다 한국 온 이유는 저는 개인적으로 한긱 음식이나 그리고 한긱 영화 한긱 두라마 좋와해서 한국에 왔습니다.'
rater.rate(s)

{'text': '저는 중국에서 왔습니다 한국 온 이유는 저는 개인적으로 한국 음식이나 그리고 한국 영화 한국 드라마 좋아해서 한국에 왔습니다.',
 'grammar': 1.0,
 'vocabulary': 1.5}

In [35]:
rater.rate("한국이 좋습니다")

{'text': '한국이 좋습니다', 'grammar': 1.0, 'vocabulary': 1.0}

## 2. Gram_Vocab_Feedback

In [36]:
class Gram_Vocab_Feedback :
    def __init__(self, correct, grammar, vocabulary, similarity) :
        self.correct = correct
        self.grammar = grammar
        self.vocabulary = vocabulary
        self.similarity = similarity
        
    def return_feedback(self, question, answer) :
        start_time = time.time()
        
        text_correct = correct.correction(answer)
        text_split = text_correct.strip().split(' ')
        text_filter = list(filter(None, text_split))
        text_join = ' '.join(text_filter)
        
        print("text_correction_time :", time.time() - start_time)  # 현재시각 - 시작시간 = 실행 시간
        start_time = time.time()
        
        recommend_return = self.similarity.recommend(question, text_join, similar_up= 0.85)
        print("text_similarity_time :", time.time() - start_time)  # 현재시각 - 시작시간 = 실행 시간
        start_time = time.time()
        
        grammar_return = [x[1] for x in self.grammar.mark_grammar(text_join)]
        grammar_return = grammar_df.iloc[grammar_return, :]
        print("text_grammar_time :", time.time() - start_time)  # 현재시각 - 시작시간 = 실행 시간
        start_time = time.time()
        
        vocabulary_return = self.vocabulary.mark_vocabulary(text_join)
        vocabulary_return = vocabulary_df.iloc[vocabulary_return, :]
        print("text_vocabulary_time :", time.time() - start_time)  # 현재시각 - 시작시간 = 실행 시간
        
        result = {
            "origin_text" : answer,
            "filtered_text" : text_join,
            "grammar" : grammar_return,
            "vocabulary" : vocabulary_return,
            "recommend" : recommend_return
        }
        return result

### instance

In [37]:
feedback = Gram_Vocab_Feedback(correct= correct_2, 
                                grammar= grammar, 
                                vocabulary= vocabulary,
                                similarity= similarity_only)

q = '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요? 왜 그 방법이 가장 도움이 되었나요?'
a = '저는 종국에서 왔습니다 한국 온 이유는 저는 개인적으로 한긱 음식이나 그리고 한긱 영화 한긱 두라마 좋와해서 한국에 왔습니다.'
feedback.return_feedback(q, a)

text_correction_time : 0.06943845748901367
질문 :  한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요? 왜 그 방법이 가장 도움이 되었나요?
학습자 답변 :  저는 중국에서 왔습니다 한국 온 이유는 저는 개인적으로 한국 음식이나 그리고 한국 영화 한국 드라마 좋아해서 한국에 왔습니다. 

0.86073524
0.85979694
0.84825516
text_similarity_time : 4.419100999832153
text_grammar_time : 0.0016260147094726562
text_vocabulary_time : 0.0009713172912597656


{'origin_text': '저는 종국에서 왔습니다 한국 온 이유는 저는 개인적으로 한긱 음식이나 그리고 한긱 영화 한긱 두라마 좋와해서 한국에 왔습니다.',
 'filtered_text': '저는 중국에서 왔습니다 한국 온 이유는 저는 개인적으로 한국 음식이나 그리고 한국 영화 한국 드라마 좋아해서 한국에 왔습니다.',
 'grammar':        등급별 번호  등급    분류   대표형   관련형   의미 국제통용 (2단계) 문법.표현 교육내용개발(1-4단계)
 전체 번호                                                                  
 9           9  1급    조사    으로     로  NaN         초급                  초급
 13         13  1급    조사    에서    서2  NaN         초급                  초급
 30         30  1급  종결어미  -습니다  -ㅂ니다  NaN         초급                  초급
 55         10  2급    조사    이나    나1  NaN         초급                  초급,
 'vocabulary':        등급별 번호  등급           어휘      품사      길잡이말 어휘교육내용개발(1-4단계)  Unnamed: 7  \
 전체 번호                                                                          
 2915     1080  3급            온     관형사      온 집안              중급         NaN   
 612       612  1급           중국      명사        나라              초급         NaN   
 521       521  1급           음

In [38]:
q = '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요? 왜 그 방법이 가장 도움이 되었나요?'
a = '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요 드라마 보고 대화 듣기 이 왜 그 방법이 가장 도움 되었어요 듣기 연습하고 새로운 어휘 나왔어요'

feed = feedback.return_feedback(q, a)

feed

text_correction_time : 0.09340786933898926
질문 :  한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요? 왜 그 방법이 가장 도움이 되었나요?
학습자 답변 :  한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요 드라마 보고 대화 듣기 이 왜 그 방법이 가장 도움 되었어요 듣기 연습하고 새로운 어휘 나왔어요 

0.97970545
0.97911143
0.9684288
text_similarity_time : 0.28658294677734375
text_grammar_time : 0.0018634796142578125
text_vocabulary_time : 0.0010030269622802734


{'origin_text': '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요 드라마 보고 대화 듣기 이 왜 그 방법이 가장 도움 되었어요 듣기 연습하고 새로운 어휘 나왔어요',
 'filtered_text': '한국어를 배우면서 가장 도움이 되었던 방법이 무엇인가요 드라마 보고 대화 듣기 이 왜 그 방법이 가장 도움 되었어요 듣기 연습하고 새로운 어휘 나왔어요',
 'grammar':        등급별 번호  등급    분류   대표형                                관련형   의미  \
 전체 번호                                                                   
 15         15  1급    조사    하고                                NaN  NaN   
 96          6  3급    조사    보고                                NaN  NaN   
 246        22  5급    표현   -었던                           -았던, -였던  NaN   
 36         36  1급  종결어미   -어2  -아2, -여2, -야3, -어요, -아요, -여요, -에요  NaN   
 59         14  2급  연결어미  -으면서                                -면서  NaN   
 241        17  5급  종결어미  -는가1                         -ㄴ가1, -은가1   의문   
 
       국제통용 (2단계) 문법.표현 교육내용개발(1-4단계)  
 전체 번호                                 
 15            초급                  초급  
 96            중급                  중급  
 246           중급        

In [39]:
feed["grammar"]

,등급별 번호,등급,분류,대표형,관련형,의미,국제통용 (2단계),문법.표현 교육내용개발(1-4단계)
전체 번호,,,,,,,,
15,15,1급,조사,하고,NaN,NaN,초급,초급
96,6,3급,조사,보고,NaN,NaN,중급,중급
246,22,5급,표현,-었던,"-았던, -였던",NaN,중급,NaN
36,36,1급,종결어미,-어2,"-아2, -여2, -야3, -어요, -아요, -여요, -에요",NaN,초급,초급
59,14,2급,연결어미,-으면서,-면서,NaN,초급,초급
241,17,5급,종결어미,-는가1,"-ㄴ가1, -은가1",의문,고급,NaN


In [40]:
feed["vocabulary"]

,등급별 번호,등급,어휘,품사,길잡이말,어휘교육내용개발(1-4단계),Unnamed: 7,표제어,동형어 번호,구분,...,전체 참고.1,검색용 이형태.1,관련어.1,의미 참고.1,다중 매체 정보.1,문형.1,문형 참고.1,뜻풀이.1,용례.1,Unnamed: 56
전체 번호,,,,,,,,,,,,,,,,,,,,,
993,258,2급,도움,명사,도움을 받다,초급,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,290,1급,보다04,부사,보다 높게,초급,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2210,375,3급,듣기,명사,듣기 평가,중급,NaN,듣기,0.0,단어,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7234,1544,5급,이04,의존명사,말하는 이,고급,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,67,1급,그02/그01,관형사/대명사,그 사람,초급,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,166,1급,대화,명사,대화를 나누다,초급,NaN,대화,0.0,단어,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1224,489,2급,새롭다,형용사,감회가 새롭다,초급,NaN,새롭다,0.0,단어,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,491,1급,왜02,부사,왜 그래?,초급,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1133,398,2급,방법,명사,사용 방법,초급,NaN,방법,0.0,단어,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
vocab_set, recommend_vocab_sets

({107, 149, 242, 452, 712, 717, 3503},
 {23,
  35,
  49,
  107,
  148,
  149,
  150,
  166,
  179,
  188,
  199,
  208,
  242,
  248,
  263,
  353,
  367,
  410,
  448,
  452,
  484,
  485,
  498,
  543,
  554,
  589,
  614,
  709,
  712,
  717,
  767,
  1021,
  1359,
  1574,
  2700,
  3186,
  3469,
  3503,
  5043,
  5186,
  5567,
  9995,
  10326})

In [54]:
text = "글쎄요"

recommend_set = similarity.recommend("여러분 나라의 학교와 한국의 학교는 무엇이 가장 다릅니까?", text)
combine_recommend = ' '.join(recommend_set)

vocab_set = set(vocabulary.mark_vocabulary(text))

recommend_vocab_sets = set(vocabulary.mark_vocabulary(combine_recommend))
        
recommend_vocab_index = list(recommend_vocab_sets - vocab_set)
recommend_set, recommend_vocab_index

({'한국 학교와 태국 학교의 다른 점은 교복인 것 같아요 제 생각에는 한국 교복이 태국 교복보다 예뻐요',
  '한국의 학교는 저희 나라 학+ 학교보다는 공부를 더 많이 시키고 교과목 이외에 어/ 다른 활동 시간이 적은 거 같습니다'},
 [5186,
  712,
  9995,
  3469,
  717,
  333,
  208,
  150,
  23,
  475,
  3675,
  35,
  1574,
  166,
  679,
  107,
  367,
  49,
  5043,
  575])

In [55]:
vocabulary_df.iloc[recommend_vocab_index]

,등급별 번호,등급,어휘,품사,길잡이말,어휘교육내용개발(1-4단계),표제어,동형어 번호,구분,품사.1,...,일본어 대역어,일본어 대역어 뜻풀이,프랑스어 대역어,프랑스어 대역어 뜻풀이,스페인어 대역어,스페인어 대역어 뜻풀이,아랍어 대역어,아랍어 대역어 뜻풀이,중국어 대역어,중국어 대역어 뜻풀이
전체 번호,,,,,,,,,,,,,,,,,,,,,
5187,1697,4급,적02,명사,적을 공격하다,중급,적,NaN,NaN,NaN,...,てき・かたき【敵】,互いに戦ったり、害を与えようとする相手。,"ennemi, adversaire, rival, concurrent",Partie adverse avec laquelle on se dispute ou ...,"enemigo, rival, contrincante, oponente",Adversario a quien se le tiene mala voluntad y...,عدوّ,الطرف الآخر الذي يحارب بعضهم بعضًا أو يحاول إي...,敌，敌人,互相争斗或意欲谋害的对方。
713,713,1급,학교,명사,학교에 다니다,초급,학교,0.0,단어,명사,...,がっこう【学校】,一定の目的、教科課程、制度などに従って、教師が児童・生徒・学生を教える機関。,"école, établissement scolaire, établissement d...",Organisme dans lequel des enseignants instruis...,"escuela, colegio",Establecimiento en el que los docentes imparte...,مدرسة,مكان يدرس فيه المدرسون طلابهم حسب الهدف المعين...,学校,按照一定的目标、教育课程、制度等，教师对学生进行教育的机构。
9996,1941,6급,점01,명사,점을 보다,고급,점,NaN,NaN,NaN,...,てん【点】。ちょぼ【点】。ぽち,小さくて丸く打った印。,"point, tache",Très petite marque de forme arrondie.,punto,Señal pequeña y redonda.,نقطة,علامة صغيرة يكتبها على شكل دائريّ,点,小而圆的标志。
3470,1635,3급,활동,명사,야외 활동,중급,활동,0.0,단어,명사,...,かつどう【活動】,体を動かして働くこと。,activité,Fait d'agir en bougeant le corps.,"Actividad , movimiento",un acto de mover el propio cuerpo para realiza...,حركة,أنْ يحرك جسما ليقوم بعمل ما,活动,活动身体。
718,718,1급,한국,명사,나라,초급,한국,0.0,단어,명사,...,かんこく【韓国】,アジア大陸の東側にある国。韓半島とその周囲の島礁から構成されていて、大韓民国（テハンミングク...,Corée (du Sud),Pays situé à l'est de l’Asie. Composé de la pé...,"Corea, Corea del Sur",País situado al este del continente asiático. ...,كوريا، جمهوريّة كوريا الجنوبيّة,دولة تقع في أقصى شرق قارة آسيا وتتكون من شبه ا...,韩国,位于亚洲大陆东部的一个国家，由朝鲜半岛及其附属岛屿构成，也被称为大韩民国。1950年朝鲜战争...
334,334,1급,생각,명사,기억,초급,생각,NaN,NaN,NaN,...,かんがえ【考え】。しこう【思考】,人間が頭を使って判断し、認識する物事。,"pensée, réflexion, idée",Fait que l'homme juge ou comprend en faisant t...,pensamiento,Reconocimiento y juicio utilizando la cabeza.,فكرة,التفكير أو الإدراك بالعقل,想，思考,人动脑子后做出判断或认知。
209,209,1급,많이,부사,많이 사다,초급,많이,0.0,단어,부사,...,おおく【多く】。たくさん【沢山】。かずおおく【数多く】。ゆたかに【豊かに】,数や量、程度などが一定の基準を超えて。,beaucoup,"(Nombre, quantité, degré, etc.) De manière à ê...","mucho, abundantemente, copiosamente","Más de un determinado número, cantidad o nivel...",كثيرا,أن يفوق العدد أو الكمية أو الدرجة غيرها بمقدار...,多,数、量、程度等超过一定标准地。
151,151,1급,다른,관형사,다른 나라,초급,다른,NaN,NaN,NaN,...,ほかの【他の】。べつの【別の】,当該すること以外の。,"autre, différent",Excepté ce qui est concerné.,"otro, demás",Además del correspondiente.,آخر,ما لا يتعلق بشأن معيّن,其他，别的,除相关之外的。
24,24,1급,같다,형용사,서로 같다,초급,같다,0.0,단어,형용사,...,おなじだ【同じだ】,互いに異ならない。,"identique, pareil, égal, même",(Deux choses ou personnes) Non différentes.,"igual, mismo, idéntico",Que no se diferencian entre sí.,يتساوي، يعادل، يوازي,لا يختلف,相同，一样，一致,彼此并无差异。


In [47]:
''.join(recommend_set)

'한국의 학교는 저희 나라 학+ 학교보다는 공부를 더 많이 시키고 교과목 이외에 어/ 다른 활동 시간이 적은 거 같습니다여러분 나라의 학교와 한국의 학교는 무엇이 가장 다릅니까 우리 나라의 학교와 한국의 학교의 다른 점은 수업 지난 시간이 다르는 것입니다 우리 나라 학교는 하루 종일 다녀야 되고 한국의 학교는 오전 반과 오후 반이 따로 있다고 들었습니다아마 공부하는 시간이 제일 다른 거 같습니다 러시아에서 어/ 아침부터 뭐/ 일곱 시 뭐/ 여덟시부터 어/ 오후 한 시까지 시간 공부하는 시간이면 한국에서는 거의 하루 종일 공부하는 거 같습니다'

In [42]:
recommend_gram_df_list

[[5186, 35, 1574, 166, 712, 107, 717, 3469, 367, 208, 49, 5043, 150, 23],
 [263,
  9995,
  148,
  149,
  150,
  35,
  554,
  3503,
  179,
  188,
  452,
  709,
  712,
  717,
  353,
  484,
  485,
  614,
  107,
  367,
  242,
  498,
  3186,
  1021],
 [2700,
  150,
  23,
  410,
  543,
  35,
  5567,
  448,
  709,
  199,
  589,
  717,
  1359,
  10326,
  485,
  367,
  3186,
  248,
  767]]